Geometric manipulations in Geopandas, projections, creating maps

are looking for three relatively large parks, which are the closest to the UCSD's CSE Department building. 

In [ ]:
%matplotlib inline 
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import folium

def closest_polys (point_lat, point_lon, polys_gdf, area_field, area_field_unit, min_area_miles, number_of_polys):
    point_df = pd.DataFrame({'Latitude': [point_lat],
     'Longitude': [point_lon]})
    point_df['Coordinates'] = list(zip(point_df.Longitude, point_df.Latitude))
    point_df['Coordinates'] = point_df['Coordinates'].apply(Point)
    point_gdf = gpd.GeoDataFrame(point_df, geometry = 'Coordinates')
    if area_field_unit == 'acre':
        polys_gdf['sqft'] = polys_gdf[area_field]*43560
    parks = polys_gdf[polys_gdf[area_field] > min_area_miles]
    parks.crs = {'init' :'epsg:2230'}
    parks2 = parks.to_crs({'init': 'epsg:4326'})
    point_gdf.crs = {'init': 'epsg:4326'}

    parks2['dist'] = parks2.distance(point_gdf.loc[0, 'Coordinates'])
    gdf_of_selected_polys_closestN = parks2.nsmallest(number_of_polys, 'dist')
    return gdf_of_selected_polys_closestN, point_gdf

In [ ]:
def map_parks (polys_gdf, choropelth_field, prop_symbol_field, num_props, point_gdf, buffers, maptitle):
    parks = polys_gdf.to_crs({'init': 'epsg:4326'})

    base = parks.plot(figsize=(15,15), column = choropelth_field, cmap='OrRd', legend = True, edgecolor = '0.8')
    largest_parks = parks.nlargest(num_props, prop_symbol_field)
    largest_parks['centroid'] = largest_parks.centroid
    largest_parks = largest_parks.set_geometry('centroid')
    park_max = largest_parks[prop_symbol_field].max()
    park_min = largest_parks[prop_symbol_field].min()
    max_size = 2000
    min_size = 50
    marker = min_size+(max_size-min_size)*(largest_parks[prop_symbol_field]/park_max)
    largest_parks.plot(ax=base, marker='o', color = 'blue', markersize= marker);
    point_gdf.plot(ax=base, marker='o', markersize= 250, color = 'black')
    convert_buff = [i/69 for i in buffers]
    for i in range(len(buffers)):
        buff = point_gdf.buffer(convert_buff[i])
        plt = buff.plot(ax = base, color = 'orange', alpha = 1/(i+2))
    plt.set_title(maptitle)
    return plt

In [ ]:
# data for testing
lat = 32.8818051
lon = -117.2357122
min_sqmiles = 0.5
area_field = 'GIS_ACRES'
area_field_unit='acre'
shpFileIn = "/datasets/dsc170sp19-public/sandiego/parks/PARKS.shp"
sd_parks = gpd.read_file(shpFileIn)
count_of_parks = 3

selected_parks,point_gdf = closest_polys(lat,lon, sd_parks, area_field, area_field_unit, min_sqmiles, count_of_parks)

selected_parks.head()
point_gdf.head()

choropelth_field = 'PARK_TYPE'
prop_symbol_field = 'GIS_ACRES'
num_props = 100
buffers = [10,20,30]
maptitle = "San Diego Parks near a point"
map_parks = map_parks(sd_parks, choropelth_field, prop_symbol_field, num_props, point_gdf, buffers, maptitle)